In [15]:
library(Signac)
library(spatstat.geom)
library(Seurat)
library(ggplot2)
library(cowplot)
library(GenomeInfoDb)
# library(matchSCore2)
#library(reticulate)
# library(EnsDb.Mmusculus.v79)
library(BSgenome.Mmusculus.UCSC.mm10)
library(EnsDb.Hsapiens.v86)
library(EnsDb.Hsapiens.v79)
library(BSgenome.Hsapiens.UCSC.hg38)
set.seed(1234)
Create_seurat <- function(rna_assay=NULL, atac_assay=NULL, sep=c("-", "-"), fragment_file=NULL, genome='hg38'){
  #
  if (is.null(rna_assay)){stop('Please provide the RNA expression matrix')}
  if (is.null(atac_assay)){stop('Please provide the ATAC expression matrix')}
  fragments <- fragment_file
  #
  proj <- CreateSeuratObject(counts = rna_assay)
  proj[['ATAC']] <- CreateChromatinAssay(counts = atac_assay, sep = sep,
                                         genome = genome, fragments = fragments)
  #
  if (genome=='hg38'){
#     annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
    annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v79)
    seqlevelsStyle(annotations) <- 'UCSC'
    Annotation(proj[["ATAC"]]) <- annotations
  }
   if (genome=='hg38'){
    annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
#     annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v79)
    seqlevelsStyle(annotations) <- 'UCSC'
    Annotation(proj[["ATAC"]]) <- annotations
  }
  if (genome=='mm10'){
    annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Mmusculus.v79)
    seqlevelsStyle(annotations) <- 'UCSC'
    Annotation(proj[["ATAC"]]) <- annotations
  }
  #
  return(proj)
}

QC_ATAC <- function(proj=proj, fragment_file=NULL, genome='hg38', group=NULL){
  DefaultAssay(proj) <- "ATAC"
  if (!is.null(fragment_file)){
    #TSS and NucleosomeSignal
    proj <- TSSEnrichment(proj)
    proj <- NucleosomeSignal(proj)
    #FRiP
    fragments <- fragment_file
    total_fragments <- CountFragments(fragments)
    total_fragments <- subset(total_fragments, total_fragments$CB %in% colnames(proj))
    row.names(total_fragments)=total_fragments$CB
    proj$fragments <- total_fragments[colnames(proj), "reads_count"]
    proj <- FRiP(object = proj,assay = 'ATAC',total.fragments = 'fragments')
  }
  #Blacklist ratio
  if (genome=='hg38'){
    proj$blacklist_fraction <- FractionCountsInRegion(proj,assay = 'ATAC',regions = blacklist_hg38)
  }
  if (genome=='mm10'){
    proj$blacklist_fraction <- FractionCountsInRegion(proj,assay = 'ATAC',regions = blacklist_mm10)
  }
  #Reident
  if(is.null(group)){Idents(proj) <- "all"}
  #
  return(proj)
}
#
QC_RNA <- function(proj=proj, genome='hg38'){
  DefaultAssay(proj) <- "RNA"
  #
  if (genome=='hg38'){
    proj[["percent.mt"]] <- PercentageFeatureSet(proj, pattern = "^MT-")
    proj[["percent.ribo"]] <- PercentageFeatureSet(proj, pattern = "^RP[SL]")
  }
  if (genome=='mm10'){
    features=grep(pattern = "^MT-", x = rownames(proj), value = TRUE,ignore.case = TRUE)
    proj[["percent.mt"]] <- PercentageFeatureSet(proj, pattern = "^mt-",features=features)
    proj[["percent.ribo"]] <- PercentageFeatureSet(proj, pattern = "^Rp[sl]")
  }
  #
  return(proj)
}


Attaching package: 'BSgenome.Hsapiens.UCSC.hg38'


The following object is masked from 'package:BSgenome.Hsapiens.UCSC.hg19':

    Hsapiens




In [87]:
setwd("../results/DORC/")

In [ ]:
train_id <- "Dataset35"
test_id <- "Dataset36"

In [89]:
RNA_count <- Read10X(data.dir = paste0("../data/",test_id,"/RNA/"), gene.column = 1)
ATAC_count <- Read10X(data.dir = paste0("../data/",test_id,"/ATAC/"), gene.column = 1)

In [90]:
KI<-grep(rownames(ATAC_count),pattern="KI")
GL<-grep(rownames(ATAC_count),pattern="GL")

[1] 68951 68952 68953 68954 68955 68956 68957 68958 68959 68960 68961 68962
[13] 68963

integer(0)

In [ ]:
ATAC_count <- ATAC_count [-c(KI,GL),]

In [95]:
# anno

In [96]:
snare <- Create_seurat(
    rna_assay = RNA_count,
    atac_assay = ATAC_count,
    sep = c("-", "-"),
    fragment_file = NULL,
    genome = "hg38"
)

Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warn

In [98]:
snare=QC_ATAC(proj = snare, fragment_file = NULL, genome = 'hg38', group = NULL)
snare=QC_RNA(proj = snare, genome = 'hg38')
snare <- subset(
      x = snare,
      subset = blacklist_fraction < 0.03 &
      nCount_RNA > 800 &
      nCount_ATAC > 500
)
snare

DefaultAssay(snare) <- 'ATAC'
snare <- RunTFIDF(snare)

DefaultAssay(snare) <- "RNA"
snare <- NormalizeData(snare)

An object of class Seurat 
105458 features across 5710 samples within 2 assays 
Active assay: RNA (36495 features, 0 variable features)
 1 other assay present: ATAC

Performing TF-IDF normalization

Warning message in RunTFIDF.default(object = GetAssayData(object = object, slot = "counts"), :
"Some features contain 0 total counts"


In [101]:
main.chroms <- standardChromosomes(BSgenome.Hsapiens.UCSC.hg38)
keep.peaks <- which(as.character(seqnames(granges(snare@assays$ATAC))) %in% main.chroms)
snare[["ATAC"]] <- subset(snare[["ATAC"]], features = rownames(snare[["ATAC"]])[keep.peaks])

In [102]:
DefaultAssay(snare) <- "ATAC"
# first compute the GC content for each peak
snare <- RegionStats(snare, genome = BSgenome.Hsapiens.UCSC.hg38)

In [105]:
annotations <- GetGRangesFromEnsDb(ensdb =EnsDb.Hsapiens.v86)
suppressWarnings(seqlevelsStyle(annotations) <- 'UCSC')
Annotation(snare[["ATAC"]]) <- annotations

Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warn

In [ ]:
snare <- LinkPeaks(
  object = snare,
  # gene.coords =  gene_coords, 
  # distance = 10e+05,
  peak.assay = "ATAC",
  expression.assay = "RNA",
  genes.use = snare@assays$RNA@counts@Dimnames[[1]]
)

Testing 17362 genes and 67226 peaks

Found gene coordinates for 13242 genes



In [ ]:
write.table(snare@assays$ATAC@links@elementMetadata,
            paste0('../results/DORC/',test_id,'.csv'),
            row.names = FALSE)